# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning
# Feature Selection

**Grupo 30: Datatouille**

**http://fdelmazo.github.io/7506-Datos/**

# AL SER UN PROCESO MUY COSTOSO EN TIEMPO Y MEMORIA, SE GUARDAN LOS RESULTADOS FINALES EN VARIABLES Y SE COMENTA TODO EL CÓDIGO

In [ ]:
features_con_saltos_progresivo = ['total_session_conversions',
 'days_to_last_conversion',
 'doy_last_event',
 'total_conversions_month_3',
 'has_conversion_month_2']

best_features_progresivo = ['timestamp_last_event',
 'total_events',
 'avg_events_per_session',
 'timestamp_last_checkout',
 'total_sessions',
 'total_session_conversions',
 'days_to_last_checkout',
 'total_checkouts_month_5',
 'doy_last_checkout',
 'woy_last_checkout',
 'total_checkouts',
 'days_to_last_conversion',
 'dom_last_event',
 'dom_last_checkout',
 'days_to_last_event',
 'doy_last_event',
 'days_to_last_viewed_product',
 'dow_last_event',
 'timestamp_last_conversion',
 'has_checkout_month_5',
 'dow_last_checkout',
 'total_session_checkouts',
 'total_checkouts_last_week',
 'total_checkouts_months_1_to_4',
 'woy_last_event',
 'doy_last_conversion',
 'dow_last_conversion',
 'dom_last_conversion']

best_features_forward = ['dow_last_conversion', 'has_conversion_last_week', 'total_conversions_month_4', 'total_session_checkouts', 'doy_last_conversion', 'timestamp_last_event', 'dow_last_checkout', 'total_checkouts', 'has_checkout', 'doy_last_checkout', 'has_checkout_month_1', 'timestamp_last_checkout', 'total_sessions', 'woy_last_event', 'has_checkout_month_5', 'avg_events_per_session']

features_con_saltos_forward = ['woy_last_checkout',
 'has_conversion_month_5',
 'has_checkout_months_1_to_4',
 'has_conversion_month_2',
 'dom_last_viewed_product']

best_features_backward = ['total_checkouts',
 'total_conversions',
 'total_events',
 'total_sessions',
 'total_session_checkouts',
 'total_session_conversions',
 'avg_events_per_session',
 'has_checkout',
 'has_conversion',
 'total_conversions_month_1',
 'total_checkouts_month_1',
 'has_checkout_month_1',
 'has_conversion_month_1',
 'total_conversions_month_2',
 'total_checkouts_month_2',
 'has_checkout_month_2',
 'has_conversion_month_2',
 'total_conversions_month_3',
 'total_checkouts_month_3',
 'has_checkout_month_3',
 'has_conversion_month_3',
 'total_conversions_month_4',
 'total_checkouts_month_4',
 'has_checkout_month_4',
 'has_conversion_month_4',
 'total_conversions_month_5',
 'total_checkouts_month_5',
 'has_checkout_month_5',
 'has_conversion_month_5',
 'total_conversions_months_1_to_4',
 'total_checkouts_months_1_to_4',
 'has_checkout_months_1_to_4',
 'has_conversion_months_1_to_4',
 'total_conversions_last_week',
 'total_checkouts_last_week',
 'has_checkout_last_week',
 'has_conversion_last_week',
 'amount_of_months_that_has_bought',
 'timestamp_last_event',
 'timestamp_last_checkout',
 'timestamp_last_conversion',
 'timestamp_last_viewed_product',
 'days_to_last_checkout',
 'days_to_last_conversion',
 'days_to_last_viewed_product',
 'doy_last_event',
 'dow_last_event',
 'dom_last_event',
 'woy_last_event',
 'doy_last_checkout',
 'dow_last_checkout',
 'woy_last_checkout',
 'doy_last_conversion',
 'dow_last_conversion',
 'dom_last_conversion',
 'woy_last_conversion',
 'doy_last_viewed_product',
 'dow_last_viewed_product',
 'dom_last_viewed_product',
 'woy_last_viewed_product']

---
---
---


In [ ]:
# import nbimporter # pip install nbimporter
# nbimporter.options['only_defs'] = False
# import pandas as pd
# import parameter_tuning as params
# import submission_framework as SF

Usando Random Forest, el algoritmo más estable de los definidos (XGBoost es poco estable) encontramos que combinación de features es la mas favorable (con la métrica Area Under Curve).

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# def random_forest(X_train, y_train, seed):
#     rf = RandomForestClassifier(**params.best_params_random_forest)
#     y_train.shape = y_train.shape[0]
#     rf.fit(X_train, y_train)
#     return rf

# model, auc = SF.full_framework_wrapper('random_forest',random_forest,verbosity=-1)

## Cumulative Importance

Se parte de una lista de todos los features ordeandos segun importancia, y se genera una lista de listas acumulativa de esto. Es decir de `[a,b,c]` se pasa a `[ [a], [a,b], [a,b,c] ]`

Esto se hace porque se esta buscando el 'codo': Los features que hacen que incremente el AUC.

In [ ]:
# feature_importances = SF.get_feature_importances('random_forest',random_forest)
# features_ordenados = feature_importances.index.tolist()
# lista_progresiva_de_cols = [features_ordenados[:i] for i in range(1,len(features_ordenados))]

# model, max_auc = SF.full_framework_wrapper('random_forest',random_forest)
# best_features_progresivo = features_ordenados
# features_con_saltos_progresivo = []

# for i, cols in enumerate(lista_progresiva_de_cols):
#     print(f'\n\nIteración {i} de {len(lista_progresiva_de_cols)}\n\n')
#     model, auc = SF.full_framework_wrapper('random_forest',random_forest,columns=cols,verbosity=1)
#     if auc > max_auc + 0.0001:
#         max_auc = auc
#         best_features_progresivo = cols
#         features_con_saltos_progresivo.append(cols[-1])
        
# best_features_progresivo

Model: random_forest - Accuracy: 0.9412 - AUC: 0.7530


Iteración 0 de 61


Model: random_forest - Columns: ['timestamp_last_event'] (AUC: 0.5557)


Iteración 1 de 61


Model: random_forest - Columns: ['timestamp_last_event', 'total_events'] (AUC: 0.5917)


Iteración 2 de 61


Model: random_forest - Columns: ['timestamp_last_event', 'total_events', 'avg_events_per_session'] (AUC: 0.5990)


Iteración 3 de 61


Model: random_forest - Columns: ['timestamp_last_event', 'total_events', 'avg_events_per_session', 'total_sessions'] (AUC: 0.6005)


Iteración 4 de 61


Model: random_forest - Columns: ['timestamp_last_event', 'total_events', 'avg_events_per_session', 'total_sessions', 'timestamp_last_checkout'] (AUC: 0.7401)


Iteración 5 de 61


Model: random_forest - Columns: ['timestamp_last_event', 'total_events', 'avg_events_per_session', 'total_sessions', 'timestamp_last_checkout', 'doy_last_checkout'] (AUC: 0.7524)


Iteración 6 de 61


Model: random_forest - Columns: ['timestamp_last_even

Model: random_forest - Columns: ['timestamp_last_event', 'total_events', 'avg_events_per_session', 'total_sessions', 'timestamp_last_checkout', 'doy_last_checkout', 'total_session_conversions', 'woy_last_checkout', 'days_to_last_checkout', 'dom_last_checkout', 'has_checkout_month_5', 'dom_last_event', 'total_checkouts_month_5', 'days_to_last_conversion', 'days_to_last_event', 'doy_last_event', 'total_checkouts', 'days_to_last_viewed_product', 'dow_last_event', 'dow_last_checkout', 'total_session_checkouts', 'total_checkouts_last_week', 'timestamp_last_conversion', 'woy_last_event', 'total_checkouts_months_1_to_4'] (AUC: 0.7596)


Iteración 25 de 61


Model: random_forest - Columns: ['timestamp_last_event', 'total_events', 'avg_events_per_session', 'total_sessions', 'timestamp_last_checkout', 'doy_last_checkout', 'total_session_conversions', 'woy_last_checkout', 'days_to_last_checkout', 'dom_last_checkout', 'has_checkout_month_5', 'dom_last_event', 'total_checkouts_month_5', 'days_to_la

Model: random_forest - Columns: ['timestamp_last_event', 'total_events', 'avg_events_per_session', 'total_sessions', 'timestamp_last_checkout', 'doy_last_checkout', 'total_session_conversions', 'woy_last_checkout', 'days_to_last_checkout', 'dom_last_checkout', 'has_checkout_month_5', 'dom_last_event', 'total_checkouts_month_5', 'days_to_last_conversion', 'days_to_last_event', 'doy_last_event', 'total_checkouts', 'days_to_last_viewed_product', 'dow_last_event', 'dow_last_checkout', 'total_session_checkouts', 'total_checkouts_last_week', 'timestamp_last_conversion', 'woy_last_event', 'total_checkouts_months_1_to_4', 'dom_last_conversion', 'has_checkout', 'doy_last_conversion', 'total_conversions', 'dow_last_conversion', 'woy_last_conversion', 'total_conversions_month_5', 'total_checkouts_month_3', 'total_checkouts_month_4', 'total_checkouts_month_2', 'total_conversions_months_1_to_4'] (AUC: 0.7519)


Iteración 36 de 61


Model: random_forest - Columns: ['timestamp_last_event', 'total_eve

Model: random_forest - Columns: ['timestamp_last_event', 'total_events', 'avg_events_per_session', 'total_sessions', 'timestamp_last_checkout', 'doy_last_checkout', 'total_session_conversions', 'woy_last_checkout', 'days_to_last_checkout', 'dom_last_checkout', 'has_checkout_month_5', 'dom_last_event', 'total_checkouts_month_5', 'days_to_last_conversion', 'days_to_last_event', 'doy_last_event', 'total_checkouts', 'days_to_last_viewed_product', 'dow_last_event', 'dow_last_checkout', 'total_session_checkouts', 'total_checkouts_last_week', 'timestamp_last_conversion', 'woy_last_event', 'total_checkouts_months_1_to_4', 'dom_last_conversion', 'has_checkout', 'doy_last_conversion', 'total_conversions', 'dow_last_conversion', 'woy_last_conversion', 'total_conversions_month_5', 'total_checkouts_month_3', 'total_checkouts_month_4', 'total_checkouts_month_2', 'total_conversions_months_1_to_4', 'has_checkout_month_4', 'amount_of_months_that_has_bought', 'has_checkout_month_3', 'has_checkout_months

Model: random_forest - Columns: ['timestamp_last_event', 'total_events', 'avg_events_per_session', 'total_sessions', 'timestamp_last_checkout', 'doy_last_checkout', 'total_session_conversions', 'woy_last_checkout', 'days_to_last_checkout', 'dom_last_checkout', 'has_checkout_month_5', 'dom_last_event', 'total_checkouts_month_5', 'days_to_last_conversion', 'days_to_last_event', 'doy_last_event', 'total_checkouts', 'days_to_last_viewed_product', 'dow_last_event', 'dow_last_checkout', 'total_session_checkouts', 'total_checkouts_last_week', 'timestamp_last_conversion', 'woy_last_event', 'total_checkouts_months_1_to_4', 'dom_last_conversion', 'has_checkout', 'doy_last_conversion', 'total_conversions', 'dow_last_conversion', 'woy_last_conversion', 'total_conversions_month_5', 'total_checkouts_month_3', 'total_checkouts_month_4', 'total_checkouts_month_2', 'total_conversions_months_1_to_4', 'has_checkout_month_4', 'amount_of_months_that_has_bought', 'has_checkout_month_3', 'has_checkout_months

Model: random_forest - Columns: ['timestamp_last_event', 'total_events', 'avg_events_per_session', 'total_sessions', 'timestamp_last_checkout', 'doy_last_checkout', 'total_session_conversions', 'woy_last_checkout', 'days_to_last_checkout', 'dom_last_checkout', 'has_checkout_month_5', 'dom_last_event', 'total_checkouts_month_5', 'days_to_last_conversion', 'days_to_last_event', 'doy_last_event', 'total_checkouts', 'days_to_last_viewed_product', 'dow_last_event', 'dow_last_checkout', 'total_session_checkouts', 'total_checkouts_last_week', 'timestamp_last_conversion', 'woy_last_event', 'total_checkouts_months_1_to_4', 'dom_last_conversion', 'has_checkout', 'doy_last_conversion', 'total_conversions', 'dow_last_conversion', 'woy_last_conversion', 'total_conversions_month_5', 'total_checkouts_month_3', 'total_checkouts_month_4', 'total_checkouts_month_2', 'total_conversions_months_1_to_4', 'has_checkout_month_4', 'amount_of_months_that_has_bought', 'has_checkout_month_3', 'has_checkout_months

['timestamp_last_event',
 'total_events',
 'avg_events_per_session',
 'total_sessions',
 'timestamp_last_checkout',
 'doy_last_checkout',
 'total_session_conversions',
 'woy_last_checkout',
 'days_to_last_checkout',
 'dom_last_checkout',
 'has_checkout_month_5',
 'dom_last_event',
 'total_checkouts_month_5',
 'days_to_last_conversion',
 'days_to_last_event',
 'doy_last_event',
 'total_checkouts',
 'days_to_last_viewed_product',
 'dow_last_event',
 'dow_last_checkout',
 'total_session_checkouts',
 'total_checkouts_last_week',
 'timestamp_last_conversion',
 'woy_last_event',
 'total_checkouts_months_1_to_4',
 'dom_last_conversion',
 'has_checkout',
 'doy_last_conversion',
 'total_conversions',
 'dow_last_conversion',
 'woy_last_conversion',
 'total_conversions_month_5',
 'total_checkouts_month_3',
 'total_checkouts_month_4',
 'total_checkouts_month_2',
 'total_conversions_months_1_to_4',
 'has_checkout_month_4',
 'amount_of_months_that_has_bought',
 'has_checkout_month_3',
 'has_checkout

##  Forward Selection

Se parte de una lista vacía y se van agregando todos los features uno por uno.

In [ ]:
# features = SF.get_full_features()

# cantidad_features = len(features)

# max_auc = 0
# features_no_usadas = features
# features_ya_usadas = []
# best_features_forward = []
# features_con_saltos_forward = []

# for i in range(cantidad_features+1):
#     print(f'\n\nIteración {i} de {cantidad_features}')
#     print(f'Lo mejor al momento: {best_features_forward} (AUC: {max_auc:.4f}) \n\n')
#     max_local_auc = 0
#     for f in features_no_usadas:
#         features_a_usar = features_ya_usadas + [f]
#         model, auc = SF.full_framework_wrapper('random_forest',random_forest,columns=features_a_usar,verbosity=1)
#         if auc > max_local_auc + 0.0001:
#             max_local_auc = auc
#             feature_a_agregar = f
        
#     features_ya_usadas += [feature_a_agregar]
#     if features_no_usadas: features_no_usadas.remove(feature_a_agregar)
    
#     if max_local_auc > max_auc + 0.0001:
#         max_auc = max_local_auc
#         best_features_forward = features_ya_usadas[:]
#         features_con_saltos_forward.append(feature_a_agregar)
        
# best_features_forward



Iteración 0 de 62
Lo mejor al momento: [] (AUC: 0.0000) 


Model: random_forest - Columns: ['total_checkouts'] (AUC: 0.7311)
Model: random_forest - Columns: ['total_conversions'] (AUC: 0.5623)
Model: random_forest - Columns: ['total_events'] (AUC: 0.4745)
Model: random_forest - Columns: ['total_sessions'] (AUC: 0.6227)
Model: random_forest - Columns: ['total_session_checkouts'] (AUC: 0.5200)
Model: random_forest - Columns: ['total_session_conversions'] (AUC: 0.6838)
Model: random_forest - Columns: ['avg_events_per_session'] (AUC: 0.4951)
Model: random_forest - Columns: ['has_checkout'] (AUC: 0.6854)
Model: random_forest - Columns: ['has_conversion'] (AUC: 0.5620)
Model: random_forest - Columns: ['total_conversions_month_1'] (AUC: 0.5066)
Model: random_forest - Columns: ['total_checkouts_month_1'] (AUC: 0.5089)
Model: random_forest - Columns: ['has_checkout_month_1'] (AUC: 0.5107)
Model: random_forest - Columns: ['has_conversion_month_1'] (AUC: 0.5066)
Model: random_forest - Columns: 

Model: random_forest - Columns: ['woy_last_checkout', 'timestamp_last_event'] (AUC: 0.6231)
Model: random_forest - Columns: ['woy_last_checkout', 'timestamp_last_checkout'] (AUC: 0.7239)
Model: random_forest - Columns: ['woy_last_checkout', 'timestamp_last_conversion'] (AUC: 0.7470)
Model: random_forest - Columns: ['woy_last_checkout', 'timestamp_last_viewed_product'] (AUC: 0.8003)
Model: random_forest - Columns: ['woy_last_checkout', 'days_to_last_event'] (AUC: 0.7405)
Model: random_forest - Columns: ['woy_last_checkout', 'days_to_last_checkout'] (AUC: 0.7400)
Model: random_forest - Columns: ['woy_last_checkout', 'days_to_last_conversion'] (AUC: 0.7555)
Model: random_forest - Columns: ['woy_last_checkout', 'days_to_last_viewed_product'] (AUC: 0.7266)
Model: random_forest - Columns: ['woy_last_checkout', 'doy_last_event'] (AUC: 0.7452)
Model: random_forest - Columns: ['woy_last_checkout', 'dow_last_event'] (AUC: 0.7891)
Model: random_forest - Columns: ['woy_last_checkout', 'dom_last_ev

Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'dow_last_checkout'] (AUC: 0.8182)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'dom_last_checkout'] (AUC: 0.8222)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'doy_last_conversion'] (AUC: 0.7939)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'dow_last_conversion'] (AUC: 0.7890)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'dom_last_conversion'] (AUC: 0.7772)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'woy_last_conversion'] (AUC: 0.8034)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'doy_last_viewed_product'] (AUC: 0.8223)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'dow_last_viewed_product'] (AUC: 0.8201)
Model: random_forest - Columns: ['woy_last_checkout', 'has_c

Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'dom_last_event'] (AUC: 0.7759)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'woy_last_event'] (AUC: 0.7921)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'doy_last_checkout'] (AUC: 0.7834)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'dow_last_checkout'] (AUC: 0.7884)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'dom_last_checkout'] (AUC: 0.7933)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'doy_last_conversion'] (AUC: 0.7762)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'dow_last_conversion'] (

Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'has_conversion_month_2', 'timestamp_last_event'] (AUC: 0.6470)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'has_conversion_month_2', 'timestamp_last_checkout'] (AUC: 0.7390)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'has_conversion_month_2', 'timestamp_last_conversion'] (AUC: 0.7572)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'has_conversion_month_2', 'timestamp_last_viewed_product'] (AUC: 0.8225)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'has_conversion_month_2', 'days_to_last_event'] (AUC: 0.7426)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'has_

Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'has_conversion_month_2', 'dom_last_viewed_product', 'total_conversions_month_4'] (AUC: 0.8106)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'has_conversion_month_2', 'dom_last_viewed_product', 'total_checkouts_month_4'] (AUC: 0.8182)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'has_conversion_month_2', 'dom_last_viewed_product', 'has_checkout_month_4'] (AUC: 0.8177)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'has_conversion_month_2', 'dom_last_viewed_product', 'has_conversion_month_4'] (AUC: 0.8209)
Model: random_forest - Columns: ['woy_last_checkout', 'has_conversion_month_5', 'has_checkout_months_1_to_4', 'has_conversion_month_2', 'dom_last_viewed_product', 'total_conversions_mont

KeyboardInterrupt: 

##  Backward Elimination

Se parte de una lista con todos los features y se van sacando uno por uno, en busqueda de cual hace que incremente un AUC una vez que se lo remueva.

In [ ]:
# features = SF.get_full_features()
# cantidad_features = len(features)

# model, max_auc = SF.full_framework_wrapper('random_forest',random_forest,verbosity=1)
# max_local_auc = max_auc
# best_features_backward = features
# features_ya_usadas = features
# features_con_saltos_backward = []

# for i in range(cantidad_features+1):
#     print(f'\n\nIteración {i} de {cantidad_features}')
#     print(f'Lo mejor al momento: {best_features_backward} (AUC: {max_auc:.4f}) \n\n')
#     for f in features_ya_usadas:
#         features_a_usar = features_ya_usadas[:]
#         features_a_usar.remove(f)
#         model, auc = SF.full_framework_wrapper('random_forest',random_forest,columns=features_a_usar,verbosity=1)
#         if auc > max_local_auc + 0.0001:
#             max_local_auc = auc
#             feature_a_borrar = f
            
#     if feature_a_borrar in features_ya_usadas: features_ya_usadas.remove(feature_a_borrar)
        
#     if max_local_auc > max_auc + 0.0001:
#         max_auc = max_local_auc
#         best_features_backward = features_ya_usadas[:]
#         features_con_saltos_backward.append(feature_a_borrar)
        
        
# # Ojo, features_con_saltos_backward se refiere a aquellos features que al removerlos se ve un incremento. 
# # Es decir, son los features que menos sirven, correr el algoritmo solo sobre esto tendría resultados muy pobres.
# best_features_backward

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'am

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'amount_of_months_that_has_bou

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'amount_of_months_that_has_bou

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'amount_of_months_that_has_bo

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'amount_of_months_that_has_bou

...

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'am

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'am

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'am

Model: random_forest - Columns: ['total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'amount_of_months_that

KeyboardInterrupt: 